In [14]:
import numpy as np
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj

In [15]:
# ---------- Step 1. Read the DEM (ASCII file) ----------
dem_file = "C:/Users/user/Studia_RSG/S2/Python_Matlab_for_Geoscience/Exam/Lubin_2024_03_27.asc"
with rasterio.open(dem_file) as dem_src:
    dem_array = dem_src.read(1)         # Read the DEM data as a 2D array
    dem_transform = dem_src.transform   # Get the affine transformation
    dem_crs = dem_src.crs               # Get the DEM's CRS

print("DEM loaded:")
print("Shape:", dem_array.shape)
print("CRS:", dem_crs)

DEM loaded:
Shape: (2398, 2261)
CRS: None


In [16]:
# ---------- Step 2. Read the point cloud shapefile ----------

points = gpd.read_file("C:/Users/user/Studia_RSG/S2/Python_Matlab_for_Geoscience/Exam/Lubin_2024_03_27_pc_t5.shp")
print("\nPoint cloud loaded:")
print(points.head())

C:\Users\user\.conda\envs\RSIP2201\Lib\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Point' is converted to 'Point Z'
  return ogr_read(



Point cloud loaded:
        Z      GPS-Time  Intensity  Scan Angle  Number of t  Number of R  \
0  140.71  3.955613e+08    31107.0         3.0            1            1   
1  140.67  3.955613e+08    30320.0         3.0            1            1   
2  140.68  3.955613e+08    27830.0         3.0            1            1   
3  140.69  3.955613e+08    33925.0         3.0            1            1   
4  140.67  3.955613e+08    33728.0         3.0            1            1   

   Classificat  User Data Edge of Fli Direction o  Point Sourc  Red Channel  \
0            2        0.0           0           0           12        25856   
1            2        0.0           0           0           12        26624   
2            2        0.0           0           0           12        25600   
3            2        0.0           0           0           12        27648   
4            2        0.0           0           0           12        29952   

   Green Chann  Blue Channe    Color           

In [18]:
# Reproject the point cloud to DEM CRS if needed
if points.crs != dem_crs:
    print("\nReprojecting point cloud to DEM CRS...")
    points = points.to_crs(dem_crs)


Reprojecting point cloud to DEM CRS...


ValueError: Must pass either crs or epsg.

In [19]:
# ---------- Step 3. Sample the DEM at point locations ----------

# Extract point coordinates (assumes the geometry type is Point)
coords = [(geom.x, geom.y) for geom in points.geometry]

# Open DEM again to sample values at the point coordinates
with rasterio.open(dem_file) as dem_src:
    # The sample() method returns an iterator of arrays (one per band)
    dem_sampled = [val[0] for val in dem_src.sample(coords)]

# Add the sampled DEM heights to the GeoDataFrame
points["DEM_H"] = dem_sampled

In [20]:
# ---------- Step 4. Calculate the height difference deltaH ----------

# Assuming the point cloud has a field "H" for its elevation
points["deltaH"] = points["H"] - points["DEM_H"]

KeyError: 'H'

In [ ]:
# ---------- Step 5. Calculate accuracy metrics ----------

deltaH = points["deltaH"].values
mean_error = np.mean(deltaH)
rmse = np.sqrt(np.mean(deltaH**2))
mae = np.mean(np.abs(deltaH))
std_dev = np.std(deltaH)

print("\nAccuracy Metrics:")
print(f"Mean Error (ME): {mean_error:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")
print(f"Standard Deviation: {std_dev:.3f}")